In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import torch

from pytrends.request import TrendReq

pd.set_option('display.max_rows', 6)

## Идеи:

* Взять отношение Media attention index по дням и оценивать функцию от него

## Цены биткоина

In [ ]:
btc_price_raw = pd.read_csv('/Users/mmenadzhiev/Desktop/Studying/Project/Estimation of the price/Data/BTC-USD.csv')
btc_price_raw.index = pd.to_datetime(btc_price_raw.Date, dayfirst=True)
btc_price_raw.drop(columns=['Date'], inplace=True)

In [ ]:
# plt.figure(figsize=(23, 9))
# btc_price_raw.Close.plot()
# plt.xticks(rotation=20)
# plt.show()

## Цена зависимой переменной

In [ ]:
eth_price_raw = pd.read_csv('/Users/mmenadzhiev/Desktop/Studying/Project/Estimation of the price/Data/ETH-USD.csv')
eth_price_raw.index = pd.to_datetime(eth_price_raw.Date, dayfirst=True)
eth_price_raw.drop(columns=['Date'], inplace=True)

## Формирование общей таблицы

In [ ]:
btc_cond = btc_price_raw.index > '2018-01-01'
btc = pd.DataFrame(btc_price_raw[btc_cond].Close)
btc.columns = ['btc']

In [ ]:
eth = pd.DataFrame(eth_price_raw.Close)
eth.columns = ['eth']

In [ ]:
btc_eth_df = pd.concat([btc, eth], join='inner', axis=1)
btc_eth_df.to_excel('btc_eth_df.xlsx')
btc_eth_df

In [ ]:
n = btc_eth_df.index.shape[0]
%store n

# Загрузим гиперпараметры
%store -r h

In [ ]:
log_btc_eth_df = pd.DataFrame(index=btc_eth_df.index)

log_btc_eth_df['log_btc']  = np.log(btc_eth_df.btc.values)
log_btc_eth_df['log_eth']  = np.log(btc_eth_df.eth.values)

log_btc_eth_df

In [ ]:
eth = torch.tensor(btc_eth_df.eth.values)
btc = torch.tensor(btc_eth_df.btc.values)

log_eth = torch.tensor(log_btc_eth_df.log_eth.values)
log_btc = torch.tensor(log_btc_eth_df.log_btc.values)

In [ ]:
btc

In [ ]:
# Сформируем матрицу X
X = torch.empty(n - h + 1, h)
for i in range(n - h + 1):
    X[i] = log_btc[i:h + i]

X = pd.DataFrame(X)
X.index = log_btc_eth_df.index[h - 1:]

columns = []
for i in range(h, 0, -1):
    columns.append(f'M_t - {i}')
    
X.columns = columns

X_index = X.index
X = X.iloc[:-1, :]
X.index = X_index[1:]

In [ ]:
log_eth_delta = pd.DataFrame(log_btc_eth_df.log_eth[h - 1:] - log_btc_eth_df.log_eth[h - 1:].shift(1)).iloc[1:, :]
log_eth_delta.columns = ['log_eth_delta']

target = log_eth_delta / np.sqrt(delta)
target.columns = ['target']

In [ ]:
# z: Z_k ~ N(0, 1)
torch.manual_seed(222)
Z = torch.distributions.normal.Normal(loc=0, scale=1).sample(sample_shape=torch.Size([n, 1]))
Z_curr = pd.DataFrame(index=X.index, data=Z[h:], columns=['Z_t'])


# print(Z.shape)
# Z

In [ ]:
# const = pd.DataFrame(np.ones_like(target), index=target.index, columns=['const'])

In [ ]:
data = pd.concat([X, Z_curr, target], axis=1)
data.to_excel('./Data/data.xlsx')
data